In [ ]:
#Make sure that you've prepared according to docs/README.md
#This notebook shows how to interact with Samantha server

In [5]:
import requests as req
import pandas as pd
import numpy as np
import time
import datetime
import random
import json
host = 'http://localhost:9100/redhat/'
dataset = 'rh'

#Change this to your data path of RedHat dataset: https://www.kaggle.com/c/predicting-red-hat-business-value
#You need to first randomly split the dataset into five folds: train-x.tsv, valid-x.tsv, test-x.tsv (x=0 to 4)
path = '/opt/kaggle/redhat/'

In [36]:
#This is a typical example of how to interact with models in Samantha server
data = {
    'modelName': 'cate-rh-model', #the name of the model in the engine configuration file
    'modelOperation': 'RESET', #could be RESET/BUILD/UPDATE/LOAD/DUMP
    'predictor': 'cate-rh' #the name of the predictor in the engine configuration file
}
recurl = host + 'predictor/model'
x = req.post(recurl, json=data)

In [ ]:
todos = ['rt', 'xgboost', 'fm', 'svdfeature', #single model
         'catnt', 'catfm', 'cate', #gbcent component
         'fm-leaf', 'svdfeature-leaf', 'xgboost-embedding', #ensemble of GBDT and matrix factorization
         'gbcent-cate', 'gbcent-catfm']  #gbcent

resultfile = path + 'results-rh.json'
fout = open(resultfile, 'a')

timefile = path + 'results-rh-time.tsv'
ftime = open(timefile, 'a')

for hit in range(0, 5):
    for what in todos:
        predictor = what + '-' + dataset
        model = predictor + '-model'
        begin = time.time()
        #training
        data = {
            'modelName': model,
            'modelOperation': 'BUILD',
            'predictor': predictor,
            'learningDaoConfig': {
                'filePath': path + 'mtrain-' + str(hit) + '.tsv',
                'separator': '\t',
                'entityDaoName': 'CSVFileEntityDAO'
            },
            'validationDaoConfig': {
                'filePath': path + 'mvalid-' + str(hit) + '.tsv',
                'separator': '\t',
                'entityDaoName': 'CSVFileEntityDAO'
            }
        }
        recurl = host + 'predictor/model'
        x = req.post(recurl, json=data)
        end = time.time()
        ftime.write('\t'.join([str(hit), predictor, str(end - begin)]) + '\n')

        #evaluating AUC
        data = {
            'predictor': predictor,
            'evaluator': 'BinaryClassificationEvaluator',
            'evaluatorDaoConfig': {
                'filePath': path + 'mtest-' + str(hit) + '.tsv',
                'separator': '\t',
                'entityDaoName': 'CSVFileEntityDAO'
            }
        }
        recurl = host + 'evaluate'
        AUC = req.post(recurl, json=data).content
        fout.write(AUC + '\n')

    data = {
        'modelName': 'cate-rh-model',
        'modelOperation': 'RESET',
        'predictor': 'cate-rh'
    }
    recurl = host + 'predictor/model'
    x = req.post(recurl, json=data)

fout.close()
ftime.close()

In [24]:
fout.close()
ftime.close()